In [20]:
import requests
import datetime
import time
import pymysql
from datetime import datetime

##  Code pour récupérer les données à intervalles de 15 min

In [21]:
import requests
import time
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go

def get_timestamp(date_str):
    """Convertit une date (string) en timestamp millisecondes"""
    dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int(dt.timestamp() * 1000)

# --- Fenêtre temporelle ---
start_date = datetime(2025, 10, 22, 0, 0, 0)
end_date = datetime(2025, 10, 23, 0, 0, 0)  # 24h pour test

# Liste de timestamps avec pas de 1h
date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime("%Y-%m-%d %H:%M:%S"))
    current_date += timedelta(hours=1)

# --- Connexion MySQL ---
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bitcoin_data",
    port=3306,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

all_data = []

# --- Récupération depuis Binance ---
for i in range(len(date_list) - 1):
    start = get_timestamp(date_list[i])
    end = get_timestamp(date_list[i + 1])

    url = f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=15m&startTime={start}&endTime={end}"
    response = requests.get(url)
    data = response.json()

    if data:
        values = []
        for kline in data:
            date = datetime.fromtimestamp(kline[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            open_price, high_price, low_price, close_price, volume = map(float, kline[1:6])
            values.append((date, open_price, high_price, low_price, close_price, volume))
            all_data.append((date, open_price, high_price, low_price, close_price, volume))

        cursor.executemany("""
            INSERT INTO btc_prices_minutes (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                open = VALUES(open),
                high = VALUES(high),
                low = VALUES(low),
                close = VALUES(close),
                volume = VALUES(volume);
        """, values)
        conn.commit()

        print(f"{len(values)} bougies insérées pour {date_list[i]} → {date_list[i+1]}")
    else:
        print(f"Aucune donnée pour {date_list[i]} → {date_list[i+1]}")

    time.sleep(1)

cursor.close()
conn.close()

# --- Création du DataFrame ---
if not all_data:
    print("Aucune donnée récupérée — le graphique ne peut pas être affiché.")
else:
    df = pd.DataFrame(all_data, columns=["date", "open", "high", "low", "close", "volume"])
    df["date"] = pd.to_datetime(df["date"])

    # --- Création du graphique chandelier ---
    fig = go.Figure(data=[go.Candlestick(
        x=df["date"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name="BTC/USDT"
    )])

    fig.update_layout(
        title="BTC/USDT — Chandeliers 15 minutes",
        xaxis_title="Date",
        yaxis_title="Prix (USDT)",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        width=1000,
        height=600
    )

    fig.show()

    print(f"{len(df)} bougies affichées — Mise à jour terminée avec succès !")


5 bougies insérées pour 2025-10-22 00:00:00 → 2025-10-22 01:00:00
5 bougies insérées pour 2025-10-22 01:00:00 → 2025-10-22 02:00:00
5 bougies insérées pour 2025-10-22 02:00:00 → 2025-10-22 03:00:00
5 bougies insérées pour 2025-10-22 03:00:00 → 2025-10-22 04:00:00
5 bougies insérées pour 2025-10-22 04:00:00 → 2025-10-22 05:00:00
5 bougies insérées pour 2025-10-22 05:00:00 → 2025-10-22 06:00:00
5 bougies insérées pour 2025-10-22 06:00:00 → 2025-10-22 07:00:00
5 bougies insérées pour 2025-10-22 07:00:00 → 2025-10-22 08:00:00
5 bougies insérées pour 2025-10-22 08:00:00 → 2025-10-22 09:00:00
5 bougies insérées pour 2025-10-22 09:00:00 → 2025-10-22 10:00:00
5 bougies insérées pour 2025-10-22 10:00:00 → 2025-10-22 11:00:00
5 bougies insérées pour 2025-10-22 11:00:00 → 2025-10-22 12:00:00
5 bougies insérées pour 2025-10-22 12:00:00 → 2025-10-22 13:00:00
5 bougies insérées pour 2025-10-22 13:00:00 → 2025-10-22 14:00:00
5 bougies insérées pour 2025-10-22 14:00:00 → 2025-10-22 15:00:00
5 bougies 

120 bougies affichées — Mise à jour terminée avec succès !


##  Code pour récupérer les données à partir de l'échelle HEURE (1)


In [ ]:
import requests
import time
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go

def get_timestamp(date_str):
    """Convertit une date (string) en timestamp millisecondes"""
    dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int(dt.timestamp() * 1000)

# --- Paramètres de la période à importer ---
start_date = datetime(2025, 10, 22, 0, 0, 0)
end_date = datetime(2025, 10, 23, 0, 0, 0)

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime("%Y-%m-%d %H:%M:%S"))
    current_date += timedelta(hours=1)  # pas de 1 heure

# --- Connexion MySQL ---
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bitcoin_data",
    port=3306,
    charset='utf8mb4',
)
cursor = conn.cursor()

# --- Récupération et insertion des données ---
for i in range(len(date_list) - 1):
    start = get_timestamp(date_list[i])
    end = get_timestamp(date_list[i + 1])
    
    url = f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=4h&startTime={start}&endTime={end}"
    response = requests.get(url)
    data = response.json()

    if data:
        values = []
        for kline in data:
            date = datetime.fromtimestamp(kline[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            open_price, high_price, low_price, close_price, volume = kline[1], kline[2], kline[3], kline[4], kline[5]
            values.append((date, open_price, high_price, low_price, close_price, volume))

        cursor.executemany("""
            INSERT INTO btc_prices_hours (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                open=VALUES(open),
                high=VALUES(high),
                low=VALUES(low),
                close=VALUES(close),
                volume=VALUES(volume);
        """, values)
        conn.commit()

        print(f"{len(values)} bougies insérées pour la période {date_list[i]} -> {date_list[i+1]}")
    else:
        print(f"Aucune donnée pour {date_list[i]} -> {date_list[i+1]}")

    time.sleep(2)

# --- Charger les données pour visualisation ---
df = pd.read_sql("SELECT date, open, high, low, close FROM btc_prices_hours ORDER BY date;", conn)
df["date"] = pd.to_datetime(df["date"])
df["open"] = df["open"].astype(float)
df["high"] = df["high"].astype(float)
df["low"] = df["low"].astype(float)
df["close"] = df["close"].astype(float)

# --- Création du graphique en chandelier ---
fig = go.Figure(data=[go.Candlestick(
    x=df["date"],
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="BTC/USDT"
)])

fig.update_layout(
    title="Chandeliers BTC/USDT — Intervalle 1h",
    xaxis_title="Date",
    yaxis_title="Prix (USDT)",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()

cursor.close()
conn.close()
print("Mise à jour terminée avec succès !")


Aucune donnée pour 2025-10-22 00:00:00 -> 2025-10-22 01:00:00
1 bougies insérées pour la période 2025-10-22 01:00:00 -> 2025-10-22 02:00:00
1 bougies insérées pour la période 2025-10-22 02:00:00 -> 2025-10-22 03:00:00
Aucune donnée pour 2025-10-22 03:00:00 -> 2025-10-22 04:00:00
Aucune donnée pour 2025-10-22 04:00:00 -> 2025-10-22 05:00:00
1 bougies insérées pour la période 2025-10-22 05:00:00 -> 2025-10-22 06:00:00
1 bougies insérées pour la période 2025-10-22 06:00:00 -> 2025-10-22 07:00:00
Aucune donnée pour 2025-10-22 07:00:00 -> 2025-10-22 08:00:00
Aucune donnée pour 2025-10-22 08:00:00 -> 2025-10-22 09:00:00
1 bougies insérées pour la période 2025-10-22 09:00:00 -> 2025-10-22 10:00:00
1 bougies insérées pour la période 2025-10-22 10:00:00 -> 2025-10-22 11:00:00
Aucune donnée pour 2025-10-22 11:00:00 -> 2025-10-22 12:00:00
Aucune donnée pour 2025-10-22 12:00:00 -> 2025-10-22 13:00:00
1 bougies insérées pour la période 2025-10-22 13:00:00 -> 2025-10-22 14:00:00
1 bougies insérées p

C:\Users\phili\AppData\Local\Temp\ipykernel_26984\343678142.py:69: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Mise à jour terminée avec succès !


##  Code pour récupérer les données à partir de l'échelle HEURE (4)

In [23]:
import requests
import time
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go

def get_timestamp(date_str):
    """Convertit une date (string) en timestamp millisecondes"""
    dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int(dt.timestamp() * 1000)

# --- Paramètres de la période à importer ---
start_date = datetime(2025, 10, 22, 0, 0, 0)
end_date = datetime(2025, 10, 23, 0, 0, 0)

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime("%Y-%m-%d %H:%M:%S"))
    current_date += timedelta(hours=4)  # pas de 4 heures

# --- Connexion MySQL ---
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bitcoin_data",
    port=3306,
    charset='utf8mb4',
)
cursor = conn.cursor()

# --- Récupération et insertion des données ---
for i in range(len(date_list) - 1):
    start = get_timestamp(date_list[i])
    end = get_timestamp(date_list[i + 1])
    
    url = f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=4h&startTime={start}&endTime={end}"
    response = requests.get(url)
    data = response.json()

    if data:
        values = []
        for kline in data:
            date = datetime.fromtimestamp(kline[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            open_price, high_price, low_price, close_price, volume = kline[1], kline[2], kline[3], kline[4], kline[5]
            values.append((date, open_price, high_price, low_price, close_price, volume))

        cursor.executemany("""
            INSERT INTO btc_prices_hours (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                open=VALUES(open),
                high=VALUES(high),
                low=VALUES(low),
                close=VALUES(close),
                volume=VALUES(volume);
        """, values)
        conn.commit()

        print(f"{len(values)} bougies insérées pour la période {date_list[i]} -> {date_list[i+1]}")
    else:
        print(f"Aucune donnée pour {date_list[i]} -> {date_list[i+1]}")

    time.sleep(2)

# --- Charger les données pour visualisation ---
df = pd.read_sql("SELECT date, open, high, low, close FROM btc_prices_hours ORDER BY date;", conn)
df["date"] = pd.to_datetime(df["date"])
df["open"] = df["open"].astype(float)
df["high"] = df["high"].astype(float)
df["low"] = df["low"].astype(float)
df["close"] = df["close"].astype(float)

# --- Création du graphique en chandelier ---
fig = go.Figure(data=[go.Candlestick(
    x=df["date"],
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="BTC/USDT"
)])

fig.update_layout(
    title="Chandeliers BTC/USDT — Intervalle 4h",
    xaxis_title="Date",
    yaxis_title="Prix (USDT)",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()

cursor.close()
conn.close()
print("Mise à jour terminée avec succès !")


1 bougies insérées pour la période 2025-10-22 00:00:00 -> 2025-10-22 04:00:00
1 bougies insérées pour la période 2025-10-22 04:00:00 -> 2025-10-22 08:00:00
1 bougies insérées pour la période 2025-10-22 08:00:00 -> 2025-10-22 12:00:00
1 bougies insérées pour la période 2025-10-22 12:00:00 -> 2025-10-22 16:00:00
1 bougies insérées pour la période 2025-10-22 16:00:00 -> 2025-10-22 20:00:00
1 bougies insérées pour la période 2025-10-22 20:00:00 -> 2025-10-23 00:00:00


C:\Users\phili\AppData\Local\Temp\ipykernel_26984\3746281229.py:69: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Mise à jour terminée avec succès !


##  Code pour récupérer les données à partir de l'échelle JOUR

In [ ]:
import requests
import time
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go

# --- Fonction de conversion en timestamp millisecondes ---
def get_timestamp(dt):
    if isinstance(dt, str):
        dt = datetime.strptime(dt, "%Y-%m-%d")
    return int(dt.timestamp() * 1000)

# --- Paramètres de la période à importer ---
start_date = datetime(2017, 8, 17)  # début BTCUSDT
end_date = datetime(2025, 10, 23)

# --- Connexion MySQL ---
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bitcoin_data",
    port=3306,
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = conn.cursor()

all_data = []

# --- Boucle par fenêtres de 1000 bougies max (~3 ans) ---
window_days = 1000
current_start = start_date

while current_start < end_date:
    current_end = min(current_start + timedelta(days=window_days), end_date)
    
    start_ts = get_timestamp(current_start)
    end_ts = get_timestamp(current_end)
    
    url = f"https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=1d&startTime={start_ts}&endTime={end_ts}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erreur API {response.status_code} pour {current_start} → {current_end}")
        time.sleep(5)
        continue

    data = response.json()
    if data:
        values = []
        for day in data:
            date = datetime.fromtimestamp(day[0] / 1000).strftime('%Y-%m-%d')
            open_price, high_price, low_price, close_price, volume = day[1], day[2], day[3], day[4], day[5]
            values.append((date, open_price, high_price, low_price, close_price, volume))
            all_data.append((date, open_price, high_price, low_price, close_price, volume))

        cursor.executemany("""
            INSERT INTO btc_prices_days (`date`, `open`, `high`, `low`, `close`, `volume`)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                `open`=VALUES(`open`),
                `high`=VALUES(`high`),
                `low`=VALUES(`low`),
                `close`=VALUES(`close`),
                `volume`=VALUES(`volume`);
        """, values)
        conn.commit()
        print(f"{len(values)} bougies insérées pour {current_start.date()} → {current_end.date()}")
    else:
        print(f"Aucune donnée pour {current_start.date()} → {current_end.date()}")

    time.sleep(1)  # pause anti-ban
    current_start = current_end

cursor.close()
conn.close()

print("Mise à jour journalière terminée avec succès !")

# --- Conversion des données en DataFrame pour Plotly ---
df = pd.DataFrame(all_data, columns=["date", "open", "high", "low", "close", "volume"])
df["date"] = pd.to_datetime(df["date"])
df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].astype(float)

# --- Création du graphique chandelier ---
fig = go.Figure(data=[go.Candlestick(
    x=df["date"],
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="BTC/USDT"
)])

fig.update_layout(
    title="BTC/USDT — Chandeliers Journaliers",
    xaxis_title="Date",
    yaxis_title="Prix (USDT)",
    xaxis_rangeslider_visible=False,
    template="plotly_dark",
    width=1200,
    height=600
)

fig.show()


500 bougies insérées pour 2017-08-17 → 2020-05-13
500 bougies insérées pour 2020-05-13 → 2023-02-07
500 bougies insérées pour 2023-02-07 → 2025-10-23
Mise à jour journalière terminée avec succès !
